In [29]:
# DAY03
f = open('input/03.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def is_symbol(c):
    return c is not None and not c.isdigit() and c != '.'


OFFSETS = [
    [-1, -1], [0, -1], [1, -1],
    [-1, 0], [1, 0],
    [-1, 1], [0, 1], [1, 1]]


def get_at(lines, x, y):
    h = len(lines)
    w = len(lines[0])
    if x >= 0 and x < w and y >= 0 and y < h:
        return lines[y][x]
    return None

def has_symbol_neighbor(lines, i, j):
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        if is_symbol(get_at(lines, x, y)):
            return True
    return False

def get_gear_neighbors(lines, i, j):
    res = set()
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        c = get_at(lines, x, y)
        if c == '*':
            res.add((x, y))
    return res

def extract_nums(lines):
    h = len(lines)
    w = len(lines[0])
    
    is_part = False
    n = 0
    num_digits = 0
    nums = []
    gears = set()
    for j in range(h):
        for i in range(w):
            c = lines[j][i]
            
            if c.isdigit():
                n = n * 10 + int(c)
                num_digits += 1
                is_part = is_part or has_symbol_neighbor(lines, i, j)
                g = get_gear_neighbors(lines, i, j)
                if g is not None:
                    gears = gears.union(g)
                
            if (not c.isdigit() or i == w - 1) and num_digits > 0:
                nums.append((n, is_part, gears))
                is_part = False
                n = 0
                num_digits = 0
                gears = set()
    return nums


def extract_gears(nums):
    gears = {}
    for _, _, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] += 1
            else:
                gears[g] = 1
                
    gears = {g: 1 for g, count in gears.items() if count == 2}
    
    for n, is_part, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] *= n
    return gears
    

nums = extract_nums(lines)
res1 = sum(n for n, is_part, _ in nums if is_part)
print(res1)

gears = extract_gears(nums)
res2 = sum(gears.values())
print(res2)

530849
84900879


In [30]:
# DAY04
f = open('input/04.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_card(line):
    line = line.split(": ")[1]
    n1, n2 = line.split("|")
    player_nums = [int(x.strip()) for x in n2.strip().split()]
    winning_nums = [int(x.strip()) for x in n1.strip().split()]
    return (winning_nums, player_nums)

def get_num_matches(card):
    common = set(card[0]).intersection(set(card[1]))
    return len(common)

def get_score(card):
    num_matches = get_num_matches(card)
    return int(2 ** num_matches / 2)

def eval_num_copies(cards):
    ncards = len(cards)
    copies = [1] * ncards
    for i in range(ncards):
        nm = get_num_matches(cards[i])
        for j in range(i + 1, min(i + 1 + nm, ncards)):
            copies[j] += copies[i]
    return copies
        
cards = [parse_card(line) for line in lines]

res1 = sum(get_score(c) for c in cards)
print(res1)

copies = eval_num_copies(cards)
res2 = sum(copies)
print(res2)
        
    

25651
19499881


In [31]:
# DAY05
text = open('input/05.txt', 'r').read()

def parse_ints(line):
    return [int(x) for x in line.split()]
    
def parse_mapping(text):
    parts = text.split('\n')
    return [parse_ints(p) for p in parts[1:] if p != '']
    
def parse_problem(text):
    parts = text.split('\n\n')
    seeds = parse_ints(parts[0].split(': ')[1])
    mappings = [parse_mapping(p) for p in parts[1:]]
    return seeds, mappings

def map_seed(seed, mappings):
    for m in mappings:
        for d, s, l in m:
            if seed >= s and seed < s + l:
                seed = d + seed - s
                break
    return seed

def map_range(range, mapping):
    ranges = set()
    ranges.add(range)
    res = []
    while len(ranges) > 0:
        r = ranges.pop()
        ranges.add(r)
        mapped = False
        for d, s, l in mapping:
            rs, rl = r
            if (s < rs + rl) and (s + l > rs):
                p1 = d + max(s, rs) - s
                p2 = d + min(s + l, rs + rl) - s
                res.append((p1, p2 - p1))
                mapped = True
                # clip the range:
                ranges.remove(r)
                if rs < s:
                    ranges.add((rs, s - rs))
                if rs + rl > s + l:
                    ranges.add((s + l, rs + rl - s - l))
                break
        if not mapped:
            break
    res += list(ranges)
    return res

def map_ranges(ranges, mappings):
    for mapping in mappings:
        res = []
        for r in ranges:
            res += map_range(r, mapping)
            ranges = res
    return ranges

seeds, mappings = parse_problem(text)

res1 = min(map_seed(s, mappings) for s in seeds)
print(res1)

ranges = [(seeds[i * 2], seeds[i * 2 + 1]) for i in range(int(len(seeds)/2))]
res2 = min(x for x, _ in map_ranges(ranges, mappings))
print(res2)

331445006
6472060


In [32]:
# DAY06
import math
from operator import mul
from functools import reduce

f = open('input/06.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_ints(line):
    return [int(x.strip()) for x in line.split()]

def parse_input(lines):
    t = parse_ints(lines[0].split(':')[1])
    d = parse_ints(lines[1].split(':')[1])
    return [(t[i], d[i]) for i in range(len(t))]

def get_roots_dist(t, d):
    det = t * t - 4 * d
    if det < 0:
        return 0
    ds = math.sqrt(det)
    r1 = math.ceil(0.5 * (ds + t))
    r2 = math.floor(0.5 * (t - ds))
    return abs(r1 - r2 - 1)
    
races = parse_input(lines)
dists = [get_roots_dist(t, d) for t, d in races]
res1 = reduce(mul, dists, 1)
print(res1)

lines2 = parse_input([l.replace(' ', '') for l in lines])
res2 = get_roots_dist(races2[0][0], races2[0][1])
print(res2)

1083852
23501589
